In [51]:
from models.Payload import RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date

#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region='Valparaiso'))
print(f"usuarios regionales: {int(q[0]['CLIENTES']):,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload())
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']=='Valparaiso'].drop(columns=['HORA','DIA','MES','ANHO'])

curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA'],as_index=False)['CLIENTES_AFECTADOS'].sum()

clientes regionales: 780,563


,NOMBRE_COMUNA,NOMBRE_EMPRESA,CLIENTES_AFECTADOS
0,Algarrobo,LITORAL,2
1,Calle Larga,CHILQUINTA,4
2,Casablanca,EDECSA,2
3,El Quisco,LITORAL,236
4,La Ligua,CGE,1
5,Papudo,CGE,3
6,Petorca,CGE,1
7,Puchuncavi,CGE,1
8,Quilpue,CHILQUINTA,83
9,San Antonio,CGE,2
